In [1]:
from coding_test.sql_commons import DbConnection

# Create CUSTOMERS Table

In [79]:
create_customer_table = """
CREATE TABLE Customers(
    CustomerId INT,
    FirstName VARCHAR(100),
    LastName VARCHAR(100),
    Email VARCHAR(100),
    City VARCHAR(100),
    State VARCHAR(100)
);
"""

with DbConnection() as db_connection:
    con, cur = db_connection
    con.execute(create_customer_table)

In [112]:
data = [
    (1, "John", "Collins", "john@example.com", "Chicago", "IL"),
    (2, "Jennifer", "McCarthy", "jennifer@example.com", "New York", "NY"),
    (3, "Anna", "Doe", "anna@example.com", "Stamford", "CT"),
    (4, "Maria", "Clark", "maria@example.com", "Hartford", "CT"),
    (5, "William", "Yang", "william@example.com", "White Plains", "NY"),
    (6, "Alex", "Muir", "alex@example.com", "New York", "NY"),
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Customers VALUES(?, ?, ?, ?, ?, ?)", data)
    con.commit()  # Remember to commit the transaction after executing INSERT.

# Create ORDERS Table

In [81]:
with DbConnection() as db_connection:
    con, cur = db_connection
    con.execute(
"""
CREATE TABLE Orders(
    OrderId INT,
    OrderQty INT,
    CustomerId INT,
    ProductId INT,
    CreatedAt DATETIME
);
""")

In [82]:
data = [
    (1, 6, 2, 1, "20190618 10:34:00 AM"),
    (2, 5, 1, 3, "20190721 11:21:00 AM"),
    (3, 6, 3, 4, "20191115 09:12:00 PM"),
    (4, 2, 4, 5, "20190911 12:04:00 AM"),
    (5, 3, 3, 2, "20190325 01:06:00 PM"),
    (6, 4, 5, 2, "20190204 05:22:00 AM"),
   
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Orders VALUES(?, ?, ?, ?, ?)", data)
    con.commit()

# Create PRODUCTS Table

In [83]:
with DbConnection() as db_connection:
    con, cur = db_connection
    con.execute(
"""
CREATE TABLE Products(
    ProductId INT,
    ProductName VARCHAR(100),
    Price NUMERIC
);
""")

In [84]:
data = [
    (1, "Shampoo", 11.99),
    (2, "Conditioner", 15.99),
    (3, "Face Wash", 5.99),
    (4, "Moisturizer", 8.99),
    (5, "Tooth Paste", 2.99)
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Products(ProductId, ProductName, Price) VALUES(?, ?, ?)", data)
    con.commit()

# Practice Question #1

Write a query to show customers who live in the state of NY and order them by their last name in ascending order.

In [2]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT FirstName, LastName, State
        FROM Customers
        WHERE State = 'NY'
        ORDER BY LastName ASC;
        """
        ):
        print(row)

('Jennifer', 'McCarthy', 'NY')
('Alex', 'Muir', 'NY')
('William', 'Yang', 'NY')


# Practice Question #2

Write a query to show the total number of orders placed by each customer and sort by highest orders.

Showing TotalOrders by CustomerId will suffice.

In [53]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT CustomerId, Count(OrderId) as TotalOrders
        FROM Orders
        GROUP BY CustomerId
        ORDER BY TotalOrders DESC;
        """
        ):
        print(row)

(2, 2)
(5, 1)
(4, 1)
(3, 1)
(1, 1)


# Practice Question #3

Write a query to show states that have more than one customer record

In [3]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT State, Count(CustomerId)
        FROM Customers
        GROUP BY State
        HAVING Count(CustomerId) > 1;
        """
        ):
        print(row)

('CT', 2)
('NY', 3)


# Create SUPPLIERS Table

In [57]:
with DbConnection() as db_connection:
    con, cur = db_connection
    con.execute(
"""
CREATE TABLE Suppliers(
    SupplierId INT,
    Name VARCHAR(100),
    Email VARCHAR(100),
    City VARCHAR(100),
    State VARCHAR(100)
);
""")

In [58]:
data = [
    (1, "ABC Group", "abc@example.com", "Chicago", "IL"),
    (2, "XYZ Group", "xyz@example.com", "New York", "NY")
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Suppliers(SupplierId, Name, Email, City, State) VALUES(?, ?, ?, ?, ?)", data)
    con.commit()

# Practice Question #4

Write a query to show the list of cities where the customers and suppliers are from

In [59]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT City FROM Customers
        UNION
        SELECT City FROM Suppliers;
        """
        ):
        print(row)

('Chicago',)
('Hartford',)
('New York',)
('Stamford',)
('White Plains',)


# Practice Question #5

Write a query to show OrderIds that have OrderQty greater than the average across all orders

In [64]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT OrderId, OrderQty, ProductId
        FROM Orders
        WHERE OrderQty > (SELECT AVG(OrderQty) FROM Orders)
        """
        ):
        print(row)

(1, 6, 1)
(2, 5, 3)
(3, 6, 4)


# Practice Question #6

Write a query to show the first order of each customer. Showing CustomerId, OrderId will suffice

In [6]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT or1.CreatedAt, or1.CustomerId, or1.OrderId
        FROM Orders AS or1
        INNER JOIN 
            (SELECT MIN(CreatedAt) as FirstOrderAt, CustomerId 
                FROM Orders 
                GROUP BY CustomerId) AS or2
            ON or1.CustomerId = or2.CustomerId
            AND or1.CreatedAt = or2.FirstOrderAt
        ORDER BY or1.CreatedAt
        """
        ):
        print(row)

('20190204 05:22:00 AM', 5, 6)
('20190325 01:06:00 PM', 3, 5)
('20190618 10:34:00 AM', 2, 1)
('20190721 11:21:00 AM', 1, 2)
('20190911 12:04:00 AM', 4, 4)


# Practice Question #7

Write a SQL query to display the full name of a state. For example, if the state is IL, shows Illinois. For this question show the full names for the three states present in the Customers table.

In [92]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT DISTINCT(State),
            CASE State
                WHEN 'CT' THEN 'Connecticut'
                WHEN 'IL' THEN 'Illinois'
                WHEN 'NY' THEN 'New York'
            END AS State_FullName
        FROM Customers
        """
        ):
        print(row)

('IL', 'Illinois')
('NY', 'New York')
('CT', 'Connecticut')


# Practice Question #8

Write a query to show the total revenue generated by each city. Include state as column in the final result set. Revenue for each product can be calculated as a multiplication of price and order amount for that respective product.

In [96]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT ROUND(SUM(Products.Price * Orders.OrderQty),2) as TotalRevenue,
               Customers.City, Customers.State
            FROM Customers
            INNER JOIN Orders ON Customers.CustomerId = Orders.CustomerId
            INNER JOIN Products ON Orders.ProductId = Products.ProductId
        GROUP BY Customers.City, Customers.State
        ORDER BY TotalRevenue DESC
        """
        ):
        print(row)

(101.91, 'Stamford', 'CT')
(71.94, 'New York', 'NY')
(63.96, 'White Plains', 'NY')
(29.95, 'Chicago', 'IL')
(5.98, 'Hartford', 'CT')


While the majority of problems can be solved with filters, joins and aggregates, you may encounter situations where using CTE or window functions may be beneficial.

For example, window functions are great when you need to aggregate without losing the individual line item information. Window functions do not output aggregates as a single row for the group but rather as a column to the original line items that you have in the table.

CTEs are great to make your queries computationally less expensive.

For example, with a query involving a WHERE and JOIN clause, SQL will process the JOIN first and then the WHERE clause. In cases where the table size is quite large, JOIN on all rows will be an expensive computation, especially if you are planning to remove those rows anyawy with the WHERE clause. In such cases, using CTE is computationally more effective where you can apply the WHERE clause first and then do the JOIN in the following statement.

In [107]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        with amt (
            OrderId, CustomerId, OrderQty, Price, Amount
        ) as (
            SELECT o.OrderId, o.CustomerId, o.OrderQty, p.Price, (p.Price * o.OrderQty) as Amount
            FROM Orders as o
            LEFT JOIN Products as p
            ON o.ProductId = p.ProductId
        ),

        totalAmt (CustomerId, TotalAmount) as (
            SELECT CustomerId, sum(Amount) as TotalAmount
            FROM amt
            GROUP BY CustomerId
        )

        SELECT c.City, c.State, CAST(ROUND(SUM(t.TotalAmount),0) AS INT) as Revenue
        FROM Customers as c
        JOIN totalAmt as t
        ON c.CustomerId = t.CustomerId
        GROUP BY City, State
        """
        ):
        print(row)

('Chicago', 'IL', 30)
('Hartford', 'CT', 6)
('New York', 'NY', 72)
('Stamford', 'CT', 102)
('White Plains', 'NY', 64)


# Practice Question #9

Write a query to show what percentage of a state's total revenue comes from each city in that state. Assume for this you have the result set named RS with columns City, State, Revenue (similar to the result set obtained as part of Q8)

In [109]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        with amt (
            OrderId, CustomerId, OrderQty, Price, Amount
        ) as (
            SELECT o.OrderId, o.CustomerId, o.OrderQty, p.Price, (p.Price * o.OrderQty) as Amount
            FROM Orders as o
            LEFT JOIN Products as p
            ON o.ProductId = p.ProductId
        ),

        totalAmt (CustomerId, TotalAmount) as (
            SELECT CustomerId, sum(Amount) as TotalAmount
            FROM amt
            GROUP BY CustomerId
        ),

        rs (City, State, Revenue) as (
            SELECT c.City, c.State, SUM(t.TotalAmount) as Revenue
            FROM Customers as c
            JOIN totalAmt as t
            ON c.CustomerId = t.CustomerId
            GROUP BY City, State
        ),

        ro (
            Revenue, StateRevenue, City, State
        ) as (
            SELECT Revenue, SUM(Revenue) OVER (PARTITION BY State) as StateRevenue, City, State
            FROM rs
        )

        SELECT 100 * Revenue / StateRevenue as CityRevenuePct, City, State
        FROM ro
        ORDER BY CityRevenuePct DESC
        """
        ):
        print(row)

(100.0, 'Chicago', 'IL')
(94.45731763833534, 'Stamford', 'CT')
(52.93598233995585, 'New York', 'NY')
(47.06401766004415, 'White Plains', 'NY')
(5.542682361664658, 'Hartford', 'CT')


# Practice Question #10

Write a query to show total number of orders placed by a customer.

If no orders have been placed by a customer, display 0

In [123]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT Customers.FirstName, COALESCE(COUNT(Orders.OrderId), 0)
        FROM Customers
        LEFT JOIN Orders ON Customers.CustomerId = Orders.CustomerId
        GROUP BY Customers.CustomerId
        """
        ):
        print(row)

('John', 1)
('Jennifer', 1)
('Anna', 2)
('Maria', 1)
('William', 1)
('Alex', 0)


# Practice Question #11

Write a query to show customers that are from the same city

In [119]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT c1.City, c1.CustomerId, c2.CustomerId
        FROM Customers AS c1
        INNER JOIN Customers AS c2
            ON c1.City = c2.City
            AND c1.CustomerId != c2.CustomerId
        """
        ):
        print(row)

('New York', 2, 6)
('New York', 6, 2)


# Practice Question #12

Write a query to show the list of cities that the customers are from

# Clean, Drop TABLES

In [78]:
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.execute("DROP TABLE Orders")
    cur.execute("DROP TABLE Customers")
    cur.execute("DROP TABLE Products")
    con.commit()

# Show TABLES

In [42]:
with DbConnection() as db_connection:
    con, cur = db_connection
    res = con.execute("SELECT name FROM sqlite_master")
    for table in res.fetchall():
        print(table)

('Persons',)
('movie',)
('Customers',)
('Orders',)
('Products',)
